In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
import json
from IPython.display import display, Markdown

In [2]:
sample_submission=pd.read_csv("../data/raw/sample_submission.csv")
specs=pd.read_csv("../data/raw/specs.csv")
test=pd.read_csv("../data/raw/test.csv")
train=pd.read_csv("../data/interim/train.csv")
train_labels=pd.read_csv("../data/raw/train_labels.csv")

Those games roughly correspond to assessments which should be predicted. Looks like activities are given not for learning, but for relaxation (like fireworks).

In [3]:
train.title[train.type=="Game"].unique()

array(['Scrub-A-Dub', 'All Star Sorting', 'Air Show', 'Crystals Rule',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Pan Balance', 'Happy Camel', 'Leaf Leader'], dtype=object)

In [4]:
train.title[train.type=="Assessment"].unique()

array(['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)',
       'Chest Sorter (Assessment)'], dtype=object)

In [5]:
pd.options.display.max_colwidth=999
games_description=pd.DataFrame(data=[
    {"game": "Chow Time", "target": "balance the scales", 
    "fields": ["scale_weight", "target_weight", "resources", "scale_contents"]},
    {"game": "All Star Sorting", "target": "sort dinosaurs by houses",
    "fields": ["dinosaurs", "houses", "dinosaurs_placed"]},
    {"game": "Dino Dive", "target": "select dinosaur(s) which raises water in pool to given level (memory + measure)", 
     "fields":["target_level", "target_water_level"]},
    {"game": "Scrub-A-Dub", "target": "find item with right size for animal several times", 
     "fields": ["round_target", "size"]},
    {"game": "Pan Balance", "target": "Balance weights",
     "fields": ["target_weight", "weights"]},
    {"game": "Dino Drink", "target": "match sizes of eggs with holes",
    "fields": ["holes"]},
    {"game": "Air Show", "target": "Move dinousaur N spaces (into target_distances)", 
    "fields": ["distance", "target_distances"]},
    {"game": "Bubble Bath", "target": "Find container with right size", 
    "fields": ["target_containers", "containers"]},
    {"game": "Crystals Rule", "target": "Measure things",
    "fields": ["group", "target_size"]},
    {"game": "Happy Camel", "target": "Find bowl with toy using scales", 
     "fields": ["bowl_id", "scale_contents"]},
    {"game": "Leaf Leader", "target": "Balance scales",
     "fields": ["target_weight", "weight"]},
    
])
games_description

,game,target,fields
0,Chow Time,balance the scales,"[scale_weight, target_weight, resources, scale_contents]"
1,All Star Sorting,sort dinosaurs by houses,"[dinosaurs, houses, dinosaurs_placed]"
2,Dino Dive,select dinosaur(s) which raises water in pool to given level (memory + measure),"[target_level, target_water_level]"
3,Scrub-A-Dub,find item with right size for animal several times,"[round_target, size]"
4,Pan Balance,Balance weights,"[target_weight, weights]"
5,Dino Drink,match sizes of eggs with holes,[holes]
6,Air Show,Move dinousaur N spaces (into target_distances),"[distance, target_distances]"
7,Bubble Bath,Find container with right size,"[target_containers, containers]"
8,Crystals Rule,Measure things,"[group, target_size]"
9,Happy Camel,Find bowl with toy using scales,"[bowl_id, scale_contents]"


In [6]:
assessment_description=pd.DataFrame(data=[
    {"game": "Mushroom Sorter (Assessment)", "target": "Sort 3 mushrooms",
     "fields": ["stumps"]},
    {"game": "Bird Measurer (Assessment)", "target": "Measure 3 birds height",
     "fields": ["bird_height", "caterpillar", "caterpillars"]},
    {"game": "Cauldron Filler (Assessment)", "target": "Match buckets with cauldrons, then choose right bucket",
     "fields": ["bucket", "cauldron", "target_bucket"]},
    {"game": "Cart Balancer (Assessment)", "target": "Balance scales", 
     "fields": ["crystals", "left", "right"]},
    {"game": "Chest Sorter (Assessment)", "target": "Order chests using pan balance",
     "fields": ["left", "right", "pillars"]}
])
assessment_description

,game,target,fields
0,Mushroom Sorter (Assessment),Sort 3 mushrooms,[stumps]
1,Bird Measurer (Assessment),Measure 3 birds height,"[bird_height, caterpillar, caterpillars]"
2,Cauldron Filler (Assessment),"Match buckets with cauldrons, then choose right bucket","[bucket, cauldron, target_bucket]"
3,Cart Balancer (Assessment),Balance scales,"[crystals, left, right]"
4,Chest Sorter (Assessment),Order chests using pan balance,"[left, right, pillars]"


In [7]:
def show_game_session(df, title, filter_cols=tuple()):
    random_row = df[df.title == title].sample(1)
    installation_id = random_row.installation_id.iloc[0]
    session_id = random_row.game_session.iloc[0]
    show_player_sessions(df, installation_id, session_id, filter_cols)


def show_player_sessions(df, installation_id, session_id=None, filter_cols=tuple()):
    df = df[df.installation_id == installation_id].copy()
    game_sessions = [session_id] if session_id else df.game_session.unique()
    for game_session in game_sessions:
        session = df[df.game_session==game_session].copy()
        display(Markdown(describe_session(game_session, session)))
        with pd.option_context('display.max_rows', 999, 'display.max_columns', 999, 
                               'display.max_colwidth', 999):
            display(format_session(session, filter_cols))


def describe_session(game_session, session):
    head = session.sort_values("event_count").head(1)
    if "version" in json.loads(head.event_data.iloc[0]):
        version = json.loads(head.event_data.iloc[0])["version"]
    else:
        version = "-"
    return f"__{game_session} {head.title.iloc[0]} {head.type.iloc[0]} {head.world.iloc[0]} v{version}__ "


def format_session(session, filter_cols):
    session = session.sort_values("event_count")
    session["timestamp"]=session["timestamp"].str[:19]
    session["event_data"] = session.event_data.apply(json.loads)
    columns = sorted(reduce(set.__or__, session.event_data.apply(lambda d: set(d.keys())).values))
    for c in columns:
        session[c] = session.event_data.apply(lambda x: x.get(c, '-'))
    session["version"]=None
    session = session.drop(["game_session", "installation_id", "event_data", "game_time", 
                            "title", "type", "world", "version"] + [c for c in filter_cols if c in session.columns], axis=1)
    session.set_index("event_count", inplace=True)
    
    return session


for game in set(train.title[train.type=="Game"].unique()):
    show_game_session(train, game, ["coordinates"])

__82a55bda928f94ac Dino Drink Game MAGMAPEAK v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,duration,has_water,holding_shell,hole_position,holes,identifier,media_type,misses,position,round,shell_size,shells,total_duration
event_count,,,,,,,,,,,,,,,,,,,,
1,2842,51311d7a,2019-09-30T00:52:23,2000,3849165.0,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-
2,2843,5be391b5,2019-09-30T00:52:26,4010,3849166.0,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-
3,2844,c6971acf,2019-09-30T00:52:28,2060,3849167.0,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-
4,2845,a29c5338,2019-09-30T00:52:28,3010,3849168.0,-,"Our competitors are getting thirsty, folks. Let's get everyone hydrated at the Hydration Station!",-,-,-,-,-,501,audio,-,-,0,-,-,6186
5,2846,6c517a88,2019-09-30T00:52:32,4070,3849169.0,-,-,-,False,False,-,-,-,-,-,-,0,-,-,-
6,2847,6c517a88,2019-09-30T00:52:33,4070,3849170.0,-,-,-,False,False,-,-,-,-,-,-,0,-,-,-
7,2848,7f0836bf,2019-09-30T00:52:35,3110,3849171.0,-,"Our competitors are getting thirsty, folks. Let's get everyone hydrated at the Hydration Station!",6380,-,-,-,-,501,audio,-,-,0,-,-,-
8,2849,a29c5338,2019-09-30T00:52:35,3010,3849172.0,-,"Watch me first, kids. First, tap an eggshell. Then, hold it under the water to fill it.",-,-,-,-,-,502_touch,animation,-,-,0,-,-,5600
9,2850,6c517a88,2019-09-30T00:52:35,4070,3849173.0,-,-,-,False,False,-,-,-,-,-,-,0,-,-,-


__13795e5537610ae4 Bubble Bath Game MAGMAPEAK v1__ 

,Unnamed: 1,event_id,timestamp,event_code,index,container_type,containers,correct,current_containers,description,duration,identifier,media_type,misses,movie_id,object,position,resources,round,target_containers,total_containers,total_duration,toy_earned
event_count,,,,,,,,,,,,,,,,,,,,,,,
1,250,1cf54632,2019-08-01T14:25:52,2000,166315.0,-,-,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
2,251,8d84fa81,2019-08-01T14:25:54,4010,166316.0,-,-,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
3,252,99abe2bb,2019-08-01T14:25:57,2080,166317.0,-,-,-,-,Peggy Peg is splashing in the bathroom and she needs your help! We're going to have a bubble party and we need your help filling the tub with water!,9933,-,-,-,Intro,-,-,-,0,-,-,-,-
4,253,99ea62f3,2019-08-01T14:26:07,2083,166318.0,-,-,-,-,Peggy Peg is splashing in the bathroom and she needs your help! We're going to have a bubble party and we need your help filling the tub with water!,9933,-,-,-,Intro,-,-,-,0,-,-,-,-
5,254,0413e89d,2019-08-01T14:26:07,3010,166319.0,-,-,-,-,Which container would you like to use? Pick one!,-,Mom_ChooseTheContainer,audio,-,-,-,-,-,0,-,-,2813,-
6,255,15eb4a7d,2019-08-01T14:26:10,3110,166320.0,-,-,-,-,Which container would you like to use? Pick one!,3060,Mom_ChooseTheContainer,audio,-,-,-,-,-,0,-,-,-,-
7,256,1beb320a,2019-08-01T14:26:11,2020,166321.0,redMug,-,-,-,-,-,-,-,-,-,-,-,"[4, 6, 8]",1,8,-,-,-
8,257,0413e89d,2019-08-01T14:26:11,3010,166322.0,-,-,-,-,Tap on the container to pour water in the tub.,-,Mom_TapTheContainer,audio,-,-,-,-,-,1,-,-,2575,-
9,258,15eb4a7d,2019-08-01T14:26:14,3110,166323.0,-,-,-,-,Tap on the container to pour water in the tub.,2816,Mom_TapTheContainer,audio,-,-,-,-,-,1,-,-,-,-


__025ce07f785229c7 Happy Camel Game CRYSTALCAVES v1.0__ 

...

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,duration,group,identifier,media_type,misses,object_type,position,resources,round,target_size,total_duration
event_count,,,,,,,,,,,,,,,,,,
1,4494,48349b14,2019-08-25T19:10:55,2000,7332274.0,-,-,-,-,-,-,-,-,-,-,0,-,-
2,4495,cc5087a3,2019-08-25T19:10:57,4010,7332275.0,-,-,-,-,-,-,-,-,-,-,0,-,-
3,4496,5154fc30,2019-08-25T19:10:59,3010,7332276.0,-,"Let's measure some big rocks with something smaller. You can move the object under the rock to figure out how long the rock is Once you know how many objects you need, tap on the right number.",-,-,instructions_touch,animation,-,-,-,-,0,-,12400
4,4497,5e3ea25a,2019-08-25T19:11:03,4070,7332277.0,-,-,-,-,-,-,-,-,-,-,0,-,-
5,4498,5e3ea25a,2019-08-25T19:11:04,4070,7332278.0,-,-,-,-,-,-,-,-,-,-,0,-,-
6,4499,5e3ea25a,2019-08-25T19:11:06,4070,7332279.0,-,-,-,-,-,-,-,-,-,-,0,-,-
7,4500,3babcb9b,2019-08-25T19:11:11,3110,7332280.0,-,"Let's measure some big rocks with something smaller. You can move the object under the rock to figure out how long the rock is Once you know how many objects you need, tap on the right number.",12766,-,instructions_touch,animation,-,-,-,-,0,-,-
8,4501,7cf1bc53,2019-08-25T19:11:11,2020,7332281.0,-,-,-,-,-,-,-,shovel,-,"[2, 3]",1,3,-
9,4502,5154fc30,2019-08-25T19:11:11,3010,7332282.0,-,How many shovels long is this rock?,-,-,howMany_shovel,animation,-,-,-,-,1,-,2933


__12398da63ccff1e5 Air Show Game TREETOPCITY v1__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,dinosaur,distance,duration,identifier,media_type,misses,round,round_prompt,target_distances,total_duration,tutorial_step
event_count,,,,,,,,,,,,,,,,,,
1,1153,15ba1109,2019-09-18T19:43:56,2000,11021848.0,-,-,-,-,-,-,-,-,0,-,-,-,-
2,1154,65abac75,2019-09-18T19:43:58,4010,11021849.0,-,-,-,-,-,-,-,-,0,-,-,-,-
3,1155,06372577,2019-09-18T19:44:05,2060,11021850.0,-,-,-,-,-,-,-,-,0,-,-,-,-
4,1156,f28c589a,2019-09-18T19:44:05,3010,11021851.0,-,"First, click the space where you want your dinosaur to start.",-,-,-,FirstClickTheSpace,animation,-,0,-,-,4366,-
5,1157,bcceccc6,2019-09-18T19:44:06,4070,11021852.0,-,-,-,-,-,-,-,-,0,-,-,-,-
6,1158,bcceccc6,2019-09-18T19:44:08,4070,11021853.0,-,-,-,-,-,-,-,-,0,-,-,-,-
7,1159,d2659ab4,2019-09-18T19:44:09,2075,11021854.0,-,-,-,-,4216,-,-,-,0,-,-,-,1
8,1160,a1bbe385,2019-09-18T19:44:09,3110,11021855.0,-,"First, click the space where you want your dinosaur to start.",-,-,4216,FirstClickTheSpace,animation,-,0,-,-,-,-
9,1161,f28c589a,2019-09-18T19:44:09,3010,11021856.0,-,"The other team will fly first. To win the gold, follow my instructions. I'll tell you where to land.",-,-,-,TheOtherTeamWillFlyFirst,animation,-,0,-,-,6966,-


__7042448bbf3921b0 Dino Dive Game MAGMAPEAK v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,dinosaur,duration,identifier,media_type,misses,round,size,target_water_level,total_duration,water_level
event_count,,,,,,,,,,,,,,,,,
1,1043,29bdd9ba,2019-09-01T01:15:50,2000,6268300.0,-,-,-,-,-,-,-,0,-,-,-,-
2,1044,87d743c1,2019-09-01T01:15:53,4010,6268301.0,-,-,-,-,-,-,-,0,-,-,-,-
3,1045,832735e1,2019-09-01T01:15:55,3010,6268302.0,-,The Claw Stars going for the gold in Dinosaur Dive!,-,-,clip_300,animation,-,0,-,-,3750,-
4,1046,ab3136ba,2019-09-01T01:15:59,3110,6268303.0,-,The Claw Stars going for the gold in Dinosaur Dive!,-,3784,clip_300,animation,-,0,-,-,-,-
5,1047,832735e1,2019-09-01T01:15:59,3010,6268304.0,-,Volume is the amount of space something takes up. Let's look at our diving team.,-,-,clip_301,animation,-,0,-,-,5433,-
6,1048,76babcde,2019-09-01T01:16:03,4070,6268305.0,-,-,-,-,-,-,-,0,-,-,-,-
7,1049,76babcde,2019-09-01T01:16:03,4070,6268306.0,-,-,-,-,-,-,-,0,-,-,-,-
8,1050,76babcde,2019-09-01T01:16:03,4070,6268307.0,-,-,-,-,-,-,-,0,-,-,-,-
9,1051,76babcde,2019-09-01T01:16:04,4070,6268308.0,-,-,-,-,-,-,-,0,-,-,-,-


In [8]:
for game in set(test.title[test.type=="Assessment"].unique())-set(assessment_description.game.values):
    show_game_session(train, game, ["coordinates"])